<a href="https://colab.research.google.com/github/tiachoi2001/ToBigs20/blob/main/Week1_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Library 와 Framework 의 차이 간단하게 서술하기. (100자 내외)

Library는 특정 기능을 가진 함수, 클래스, 모듈로 이루어져 사용자가 가져와 쓸 수 있게 도와준다. Framework는 조금 더 고급 툴로 해당 프로젝트의 전체적인 구조와 아키텍쳐를 제공한다. 둘 중 하나가 더 우위에 있지는 않지만 framework는 프로젝트의 전체적인 흐름을 관장하는 반면 library는 특정 기능이 필요할 때 빌려와서 쓰기 떄문에 전체적인 흐름에는 영향을 적게 미친다.

2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하기. (200자 내외)

머신러닝은 딥러닝을 포함하는 개념이다. 다르게 말하면 머신러닝의 세부 분야 중 딥러닝이 있는 것이다. 이 두 개념의 가장 큰 차이는 머신러닝은 사람이 특징을 추출하고 분류하는 과정에서 개입을 하는데 딥러닝은 인공신경망의 구축을 통해 스스로 학습한다는 점이다. 사람의 개입이 적고 다량의 데이터를 학습할 수 있다는 점에서 딥러닝은 빅데이터에 적합한 기술이라고 할 수 있다.

3. 주석으로 설명하기

Random Forest Example

In [ ]:
### matplotlib 한글 폰트 깨짐 방지
### 이 셀 실행시키고 런타임->런타임 다시 시작 한 다음에
### 이 셀은 건너뛰고 아래부터 실행 시키면 됨.

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
# 드라이브 mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# 데이터 불러오기
df = pd.read_csv("/content/drive/MyDrive/AI/bundang.csv")
df = df.drop(columns=['시군구','단지명','건축년도'])
df.head()

In [ ]:
# NaN값 개수
df.isna().sum()

In [ ]:
# 거래금액 히스토그램
plt.hist(df['거래금액'],bins=100)

In [ ]:
# log값을 취해서 히스토그램
df['거래금액_log'] = np.log(df['거래금액']+1)
df = df.drop(columns=['거래금액'])
plt.hist(df['거래금액_log'],bins=100)

In [ ]:
# 사이킷런 모듈 가져오기
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# 타깃(Y)과 인풋(X) 설정
Y = df['거래금액_log'].values
Y = Y.astype('float32')
X = df.drop(columns=['거래금액_log'])
X = X.astype('float32')

# 데이터셋을 학습 데이터와 테스트 데이터로 나눔
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state=42)

# RandomForestRegressor 모델 생성
RF_model = RandomForestRegressor(n_estimators=2000, random_state=42)
# 학습
RF_model.fit(X_train,Y_train)

In [ ]:
# 예측
Train_Prediction = (RF_model.predict(X_train))
Test_Prediction = (RF_model.predict(X_test))

# 학습 데이터와 테스트 데이터 각각 회귀 모델 성능 평가 지표인 MAE와 MAPE 구하기
RF_Train_MAE = round(np.mean(np.abs(Y_train-Train_Prediction)),3)
RF_Train_MAPE = round((np.mean(np.abs((Y_train-Train_Prediction)/(Y_train)))*100),3)

RF_Test_MAE = round(np.mean(np.abs(Y_test-Test_Prediction)),3)
RF_Test_MAPE = round((np.mean(np.abs((Y_test-Test_Prediction)/(Y_test)))*100),3)

print("Mean Absolute Error for Train Set is : {:.3f}".format(RF_Train_MAE))
print("Mean Absolute Percent Error for Train Set is : {:.3f} %".format(RF_Train_MAPE))
print("Mean Absolute Error for Test Set is : {:.3f}".format(RF_Test_MAE))
print("Mean Absolute Percent Error for Test Set is : {:.3f} %".format(RF_Test_MAPE))

In [ ]:
# log 값 취한 것을 다시 되돌림 (실제 / 예측값)
Original_Predictions = (np.round(np.exp(Test_Prediction+1),-2)).astype(int)
print(Original_Predictions[:10])
Original_Values = (np.round(np.exp(Y_test+1),-2)).astype(int)
print(Original_Values[:10])
data = {'Original':  Original_Values,'Prediction': Original_Predictions}
final = pd.DataFrame(data)
final

In [ ]:
# RF 모델 feature importances 불러와서 bar차트로 표
sorted_idx = RF_model.feature_importances_.argsort()
plt.barh(X.columns[sorted_idx], RF_model.feature_importances_[sorted_idx])
plt.xlabel("Random Forest Feature Importance")

# XGBoost

In [ ]:
# xgboost를 사용해서 같은 것을 반복
import xgboost

# 학습 데이터와 테스트 데이터로 나누
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state=41)
XGB_model = xgboost.XGBRegressor(n_estimators=2000, random_state=41)

# 학습
XGB_model.fit(X_train,Y_train)

In [ ]:
# 예측
Train_Prediction = (XGB_model.predict(X_train))
Test_Prediction = (XGB_model.predict(X_test))

# MAE와 MAPE 구하기
XGB_Train_MAE = round(np.mean(np.abs(Y_train-Train_Prediction)),3)
XGB_Train_MAPE = round((np.mean(np.abs((Y_train-Train_Prediction)/(Y_train)))*100),3)

XGB_Test_MAE = round(np.mean(np.abs(Y_test-Test_Prediction)),3)
XGB_Test_MAPE = round((np.mean(np.abs((Y_test-Test_Prediction)/(Y_test)))*100),3)

print("Mean Absolute Error for Train Set is : {:.3f}".format(XGB_Train_MAE))
print("Mean Absolute Percent Error for Train Set is : {:.3f} %".format(XGB_Train_MAPE))
print("Mean Absolute Error for Test Set is : {:.3f}".format(XGB_Test_MAE))
print("Mean Absolute Percent Error for Test Set is : {:.3f} %".format(XGB_Test_MAPE))

In [ ]:
# feature importance 표시하기
xgboost.plot_importance(XGB_model)

DCGAN Example

In [ ]:
# 필요한 라이브러리 import
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random
import torch
import torch.nn as nn
from torch.utils import data

import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [ ]:
# GPU 활용이 가능하면 사용, 불가능하면 CPU 사용
GPU_NUM = 0
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available else 'cpu')
torch.cuda.set_device(device)
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())
print(torch.__version__)

In [ ]:
# GAN 모델의 discriminator 클래스 생성
class Discriminator(nn.Module):
    def __init__(self,channels_img,features_d):
        # 부모 클래스 상속
        super(Discriminator,self).__init__()
        # NN layer 설정
        self.disc=nn.Sequential(
            nn.Conv2d(channels_img,features_d,kernel_size=4,stride=2,padding=1),
            nn.LeakyReLU(0.2),
            self._block(features_d,features_d*2,4,2,1),
            self._block(features_d*2,features_d*4,4,2,1),
            self._block(features_d*4,features_d*8,4,2,1),
            nn.Conv2d(features_d*8,1,kernel_size=4,stride=2,padding=1),
            nn.Sigmoid(), )

    def _block(self, in_channels, out_channels, kernel_size,stride,padding):
        return nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding,bias=False,),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),)

    def forward(self,x):
        return self.disc(x)

# GAN 모델의 generator 클래스 생성
class Generator(nn.Module):
    def __init__(self,z_dim,channels_img,features_g):
        super(Generator,self).__init__()
        self.gen = nn.Sequential(
            self._block(z_dim,features_g*16,4,1,0),
            self._block(features_g*16,features_g*8,4,2,1),
            self._block(features_g*8,features_g*4,4,2,1),
            self._block(features_g*4,features_g*2,4,2,1),
            nn.ConvTranspose2d(
                features_g*2, channels_img,kernel_size=4,stride=2,padding=1,
            ),
            nn.Tanh(),
        )
    def _block(self,in_channels,out_channels,kernel_size,stride,padding):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels,out_channels,kernel_size,stride,padding,bias=False,),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
    def forward(self,x):
        return self.gen(x)

# 모델 가중치 초기화
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m,(nn.Conv2d,nn.ConvTranspose2d,nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data,0.0,0.02)

In [ ]:
# 이미지 display 함수
def show_image(dataset, times):
    for i in range(times):
        t = random.randint(1,len(dataset))
        temp_image = dataset.data[t]
        temp_image = np.array(temp_image, dtype='float')
        pixels = temp_image.reshape((28, 28))
        plt.figure(figsize=(2,2))
        plt.imshow(pixels, cmap='gray')
        plt.show()

In [ ]:
# 데이터셋 불러오기
dataset = datasets.MNIST(root="dataset/",train=True,download=True)

# 이미지 3개 display
show_image(dataset,3)

In [ ]:
# hyperparameter 설정
LEARNING_RATE = 3e-4
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNELS_IMG = 1
Z_DIM = 100
NUM_EPOCHS=40
FEATURES_DISC=64
FEATURES_GEN=64

In [ ]:
# 데이터셋 설정
dataset = datasets.MNIST(root='dataset/',download=True,train=True,transform=transforms)

# 타킷 설정
SPLIT={0:[],1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[]}
for i in range(len(dataset.targets)):
    SPLIT[dataset.targets[i].item()].append(i)

# 데이터 split (타킷)
def split(data,target):
    data.data = data.data[SPLIT[target]]
    data.targets = data.targets[SPLIT[target]]

# transform pipline 생성
Transforms = transforms.Compose([
                                 transforms.Resize(IMAGE_SIZE),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.5 for _ in range(CHANNELS_IMG)],[0.5 for _ in range(CHANNELS_IMG)]),
])

# transform 적용
one_data = datasets.MNIST(root="dataset/",train=True,transform=Transforms,download=True)
print(one_data.targets)
print(len(one_data))


### 이 한 줄을 주석처리시키면
### 60000개 데이터 전부 사용해서 학습을 시킬 수 있다.
# split(one_data,3)

# 이미지 display
show_image(one_data,3)
print(one_data.targets)
print(len(one_data))

In [ ]:
# DataLoader 생성
One_Data_Loader=DataLoader(one_data,batch_size=BATCH_SIZE, shuffle=True)
# generator와 discriminator 생성 (device로 배정)
One_gen = Generator(Z_DIM, CHANNELS_IMG , FEATURES_GEN).to(device)
One_disc = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)
# weight 초기화
initialize_weights(One_gen)
initialize_weights(One_disc)
# optimizer 설정
opt_gen = optim.Adam(One_gen.parameters(), lr = LEARNING_RATE, betas=(0.5,0.999))
opt_disc = optim.Adam(One_disc.parameters(), lr = LEARNING_RATE, betas=(0.5,0.999))
# loss 함수 설정
criterion = nn.BCELoss()

# generator와 discriminator 학습
One_gen.train()
One_disc.train()
Start_Time = time.time()

for epoch in range(NUM_EPOCHS):
		### 이 아래의 for 문 한 줄에서 y 값을 사용하고 있지 않음을 알 수 있습니다.
    for batch_idx, (real,_) in enumerate(One_Data_Loader):

        real = real.to(device)
        noise = torch.randn((BATCH_SIZE,Z_DIM,1,1)).to(device)
        fake = One_gen(noise)

        disc_real = One_disc(real).reshape(-1)
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))

        disc_fake = One_disc(fake).reshape(-1)
        loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))

        loss_disc = (loss_disc_real+loss_disc_fake)/2

        One_disc.zero_grad()
        loss_disc.backward(retain_graph=True)
        opt_disc.step()

        output = One_disc(fake).reshape(-1)
        loss_gen = criterion(output,torch.ones_like(output))
        One_gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

    # generator로 생성한 이미지 show
    if (epoch+1)%(max(1,NUM_EPOCHS//5))==0:
        Z = torch.randn(100,100,1,1).to(device)
        out_gen = One_gen(Z)
        Image_Loader=DataLoader(out_gen,batch_size=64, shuffle=True)
        real_batch = next(iter(Image_Loader))
        plt.figure(figsize=(8,8))
        plt.axis("off")
        plt.title("Fake Images in Epoch {}".format(epoch+1))
        plt.imshow(np.transpose(vutils.make_grid(real_batch.to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))
        plt.show()

    # 학습 시간 프린트
    print("Progress : [ {} / {} ], Time : {:.3f} (min) ".format(epoch+1,NUM_EPOCHS,(time.time()-Start_Time)/60) )